In [1]:
import os
from datetime import datetime

#numerical libs
import math
import numpy as np
import random
import PIL

import matplotlib
matplotlib.use('TkAgg')
#matplotlib.use('WXAgg')
#matplotlib.use('Qt4Agg')
#matplotlib.use('Qt5Agg') #Qt4Agg
print(matplotlib.get_backend())

# torch libs
import torch
import torchvision.transforms as transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.nn.parallel.data_parallel import data_parallel


# std libs
import collections
import copy
import numbers
import inspect
import shutil
from timeit import default_timer as timer

import csv
import pandas as pd
import pickle
import glob
import sys
from distutils.dir_util import copy_tree
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sklearn.preprocessing

TkAgg


In [2]:
SEED = int(time.time()) 
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
print ('\tset random seed')
print ('\t\tSEED=%d'%SEED)

	set random seed
		SEED=1530977844


In [3]:
torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
torch.backends.cudnn.enabled   = True
print ('\tset cuda environment')
print ('\t\ttorch.__version__              =', torch.__version__)
print ('\t\ttorch.version.cuda             =', torch.version.cuda)
print ('\t\ttorch.backends.cudnn.version() =', torch.backends.cudnn.version())
try:
    print ('\t\tos[\'CUDA_VISIBLE_DEVICES\']     =',os.environ['CUDA_VISIBLE_DEVICES'])
    NUM_CUDA_DEVICES = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))
except Exception:
    print ('\t\tos[\'CUDA_VISIBLE_DEVICES\']     =','None')
    NUM_CUDA_DEVICES = 1

print ('\t\ttorch.cuda.device_count()      =', torch.cuda.device_count())
print ('\t\ttorch.cuda.current_device()    =', torch.cuda.current_device())

	set cuda environment
		torch.__version__              = 0.4.0
		torch.version.cuda             = 9.0.176
		torch.backends.cudnn.version() = 7102
		os['CUDA_VISIBLE_DEVICES']     = None
		torch.cuda.device_count()      = 1
		torch.cuda.current_device()    = 0


In [4]:
class EventDataset(Dataset):
    def __init__(self, event='1000', transform=None):
        super(EventDataset,self).__init__()
        self.transform=transform
        self.path='/home/alexanderliao/data/Kaggle/competitions/trackml-particle-identification/train/event00000%s'%event
        self.size=1
        self.hits=pd.read_csv(self.path+'-hits.csv')
        self.cells=pd.read_csv(self.path+'-cells.csv')
        self.particles=pd.read_csv(self.path+'-particles.csv')
        self.truth=pd.read_csv(self.path+'-truth.csv')

    def __getitem__(self, index):
        return 1

    def __len__(self):
        return len(self.hits)

In [5]:
from trackml.dataset import load_event

In [6]:
data=EventDataset()
data.hits


,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.4099,-7.163700,-1502.5,7,2,1
1,2,-55.3361,0.635342,-1502.5,7,2,1
2,3,-83.8305,-1.143010,-1502.5,7,2,1
3,4,-96.1091,-8.241030,-1502.5,7,2,1
4,5,-62.6736,-9.371200,-1502.5,7,2,1
5,6,-57.0687,-8.177770,-1502.5,7,2,1
6,7,-73.8723,-2.578900,-1502.5,7,2,1
7,8,-63.8535,-10.868400,-1502.5,7,2,1
8,9,-97.2548,-10.889100,-1502.5,7,2,1
9,10,-90.2929,-3.269370,-1502.5,7,2,1


In [7]:
data.truth.drop(['hit_id'],axis=1)

,particle_id,tx,ty,tz,tpx,tpy,tpz,weight
0,0,-64.4116,-7.164120,-1502.5,250710.000000,-149908.000000,-956385.000000,0.000000
1,22525763437723648,-55.3385,0.630805,-1502.5,-0.570605,0.028390,-15.492200,0.000010
2,0,-83.8280,-1.145580,-1502.5,626295.000000,-169767.000000,-760877.000000,0.000000
3,297237712845406208,-96.1229,-8.230360,-1502.5,-0.225235,-0.050968,-3.702320,0.000008
4,418835796137607168,-62.6594,-9.375040,-1502.5,-0.281806,-0.023487,-6.573180,0.000009
5,108087696726949888,-57.0856,-8.189710,-1502.5,-0.401129,-0.035276,-10.466900,0.000008
6,968286151951515648,-73.8608,-2.575860,-1502.5,-0.442662,-0.036969,-9.130100,0.000007
7,954766419537428480,-63.8512,-10.875400,-1502.5,-0.670459,-0.092609,-15.540700,0.000011
8,707072769359085568,-97.2489,-10.906700,-1502.5,-0.279789,-0.062143,-4.412920,0.000008
9,67554956483231744,-90.2763,-3.243970,-1502.5,-0.251752,-0.037138,-4.249220,0.000006


In [22]:
def plot_track_3d(df, transformer_list, clusterer_list=(), n_tracks=20, cutoff=3, verbose=False):
    """
    :param df: Pandas DataFrame containing hit coordinate information
    must have the following columns: particle_id, x, y, z

    :param transformer_list: list of transformer functions.
    transformer function: df -> a numpy array of shape [n_samples, 3]

    :param clusterer_list: a list of clusterer objects; each of them will be used to transform the new dataset
    example: [DBSCAN(eps=0.01), DBSCAN(eps=0.1)]

    :param n_tracks: the number of particles/tracks to display

    :param cutoff: particles with less than or equal to cutoff hits will be ignored.

    :param verbose: if set to True, print the particle id and the coordinate matrix in the console.
    """

    for c in ["tx", "ty", "tz", "particle_id"]:
        if c not in df.columns:
            raise ValueError("[plot_track_3d] input DataFrame does not contain {}".format(c))

    df = df.copy()

    def plot_track(sub_df):
        sub_df = sub_df.sort_values(by="tz")
        ax.plot(sub_df.v1.values, sub_df.v2.values, sub_df.v3.values, ".-")
        return sub_df

    track_size = df.groupby("particle_id", sort=False)["tx"].agg("count")  # particle id -> track size
    track_size.drop(0, axis=0, inplace=True)  # ignore noisy track
    particle_list = track_size[track_size > cutoff].index  # ignore small tracks (track size <= cutoff)
    particle_list = np.random.choice(particle_list, size=min(n_tracks, particle_list.size), replace=False)

    selected_idx = df.particle_id.isin(particle_list)  # get boolean mask

    fig = plt.figure()
    i = 1

    #for transformer in transformer_list:  # iterate over transformers
    ax = fig.add_subplot(1, len(clusterer_list) + 1, i, projection='3d')
    # transform the dataset using transformer
    new_array = df
    df.loc[:, "v1"] = new_array[:, 0]
    df.loc[:, "v2"] = new_array[:, 1]
    df.loc[:, "v3"] = new_array[:, 2]

    df_agg = df.loc[selected_idx, :].groupby("particle_id", sort=False)  # aggregate selected df by particle id
    df_agg.apply(plot_track)
    ax.set_xlabel("v1"), ax.set_ylabel("v2"), ax.set_zlabel("v3")

    i += 1  # increment the subplot index

    for clusterer in clusterer_list:
        ax = fig.add_subplot(len(transformer_list), len(clusterer_list) + 1, i, projection='3d')
        df.loc[:, "cluster_id"] = clusterer.fit_predict(new_array)
        df_agg = df.loc[selected_idx, :].groupby("cluster_id", sort=False)  # aggregate selected df by particle id
        df_agg.apply(plot_track)
        ax.set_xlabel("v1"), ax.set_ylabel("v2"), ax.set_zlabel("v3")
        i += 1
    plt.show()


In [23]:
plot_track_3d(data.truth,transformer_list=None)

TypeError: unhashable type: 'slice'